In [ ]:
import arcgis
from arcgis.gis import GIS
from arcgis.apps import storymap
from arcgis.mapping import WebMap
from getpass import getpass
from arcgis.features import FeatureLayer, Feature, FeatureLayerCollection
import pandas as pd
from datetime import datetime
import datetime as dt
import os

In [ ]:
# Create a connection to your portal for publishing (enter your ArcGIS Online 
#  password in the textbox that appears, then hit 'Enter' on your keyboard)
username = " "
gis = GIS("https://www.arcgis.com", username, getpass())

In [ ]:
#Find storymaps to edit
find_storymaps = gis.content.search("title:Community Storymap", item_type="Web Mapping Application", max_items=500, outside_org=False)

In [ ]:
for f in find_storymaps:
    if f.title != "Alaska DCRA Community Storymaps" and not f.title.startswith('Communit') and 'Communit' in str(f.title):
        print(f.title, f.id)
        csmap = storymap.JournalStoryMap(gis.content.get(str(f.id)))
        sections = csmap.properties["values"]["story"]["sections"]
        panel_util = sections[9]
        if str(panel_util["title"]) == '<span style="font-size:26px">Utilities</span>' or str(panel_util["title"]) == '<span style="font-size:26px;">Utilities</span>':
            community = f.title.replace(" Community Storymap", "")
            if community.startswith("City & Borough of "):
                community = community.replace("City & Borough of ", "")
            if community.startswith("Municipality of "):
                community = community.replace("Municipality of ","")
            if community.endswith('Borough'):
                community = community.replace(" Borough","")
            if community.endswith('(Unincorporated Community)'):
                community = community.replace(' (Unincorporated Community)','')
            query = "CommunityName = '" + str(community.replace("'","''")) + "' OR CommunityName = '" + str(community.replace("'","''")) + " CDP'"

            #PCE Program/Electric Utilities (final variable to embed is elec_tbl)
            pce_base = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/Services/CDO_Utilities/MapServer/4')
            pce_data = pce_base.query(where="CommunityName = '" + str(community.replace("'","''")) + "' AND DataYear = '2016'",out_fields="*").df
            pwrsrc_base = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/Services/CDO_Utilities/MapServer/5')
            pwrsrc_data = pwrsrc_base.query(where="CommunityName = '" + str(community.replace("'","''")) + "'").df
            if str(pwrsrc_data) == 'Empty DataFrame\nColumns: []\nIndex: []':
                embed_pwrsrc = '<i> No data.</i>'
            else:
                embed_pwrsrc = '<span style="font-size:14px">' + str(pwrsrc_data.PowerSource[0]) + '</span>'
            if str(pce_data) != 'Empty DataFrame\nColumns: []\nIndex: []':
                provider = pce_data.EntityName[0]
                price_fuel = pce_data.AveragePriceOfFuelPerGallon[0]
                res_rate = pce_data.EffectiveResidentialRate[0]
                PCE_disb = pce_data.YearToDatePCEDisbursements[0]
                PCE_disb = '{:,}'.format(PCE_disb)
            else:
                provider = '<i>No data.</i>'
                price_fuel = '<i>No data.</i>'
                res_rate = '<i>No data.</i>'
                PCE_disb = '<i>No data.</i>'
            elec_tbl = '<table border="1"><tbody><tr><td width="50%"><b><span style="font-size:14px">Power Source</span></b></td><td width="50%"><span style="font-size:14px">' + str(embed_pwrsrc) + '</span></td></tr><tr><td width="50%"><b><span style="font-size:14px">PCE Provider</span></b></td><td width="50%"><span style="font-size:14px">' + str(provider) + '</span></td></tr><tr><td width="50%"><span style="font-size:14px"><b>Residential Rate</b></span></td><td><span style="font-size:14px">$' + str(res_rate) + '</span></td></tr><tr><td width="50%"><b><span style="font-size:14px">PCE Disbursements YTD</span></b></td><td width="50%"><span style="font-size:14px">$' + str(PCE_disb) + '</span></td></tr></tbody></table>'

        #Fuel survey (final variable to embed is fuel_tbl)
            fs_query = "CommunityName = '" + str(community.replace("'","''")) + "' AND ReportingYear = 2019 AND ReportingPeriod = 'Jan'" 
            gas_base = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/Services/CDO_Utilities/MapServer/6')
            gas_data = gas_base.query(where=fs_query, out_fields="*").df
            if str(gas_data) != 'Empty DataFrame\nColumns: []\nIndex: []':
                gasprice = '<span style="font-size:14px">$' + str(gas_data.GasRetailGal[0]) + '</span>'
                gas_prov = '<span style="font-size:14px">' + str(gas_data.EntityName[0]) + '</span>'
                period = '<span style="font-size:14px">' + str(gas_data.ReportingPeriod[0]) + ", " + str(gas_data.ReportingYear[0]) + '</span>'
            else:
                gasprice = '<i> No data. </i>'
                period = ''
            fuel_base = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/Services/CDO_Utilities/MapServer/7')
            fuel_data = fuel_base.query(where=fs_query, out_fields="*").df
            if str(fuel_data) != 'Empty DataFrame\nColumns: []\nIndex: []':
                fuelprice = '<span style="font-size:14px">$' + str(fuel_data.HFRetailGal[0]) + '</span>'
                fuel_prov = '<span style="font-size:14px">' + str(fuel_data.EntityName[0]) + '</span>'
            else:
                fuelprice = '<i> No data. </i>'
            if gasprice == '<i> No data. </i>' and fuelprice == '<i> No data. </i>': #and delivery == ' No data. ':
                fuel_tbl = '<span style="font-size:14px">This community is not one of the communities tracked by the <a href="https://www.commerce.alaska.gov/web/dcra/researchanalysis/fuelpricesurvey.aspx">DCRA Fuel Survey</a>.</span>'
            else:
                fuel_tbl = '<table border="1"><tbody><tr><td width="50%"><span style="font-size:14px"><b>Retail Gas Price</b></span></td><td>' + str(gasprice) + '</td></tr><tr><td width="50%"><span style="font-size:14px"><b>Gas Vendor</b></span></td><td width="50%">' + str(gas_prov) + '</td></tr><tr><td width="50%"><span style="font-size:14px"><b>Retail Heating Fuel Price</b></span></td><td width="50%">' + str(fuelprice) + '</td></tr><tr><td width="50%"><span style="font-size:14px"><b>Heating Fuel Vendor</b></span></td><td>' + str(fuel_prov) + '</td></tr></tbody></table>'

        #Water/Landfill Utilities
            #Water treatment
            h2o_treat_base = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/Services/CDO_Utilities/MapServer/8')
            h2o_t = h2o_treat_base.query(where=query, out_fields="*")
            if str(h2o_t.df) != 'Empty DataFrame\nColumns: []\nIndex: []':
                h2o_t_tbl = ['<table border="1"><th width="20%">Water Treatment System Facility</th><th width="25%">Primary Water Source</th><th width="20%">Number of Residents Served</th><th width="20%">Water Treatment System (AK Dept. of Environmental Conservation)</th><tbody>']
                facilities = h2o_t.df.FacilityName
                for f in list(set(facilities)):
                    h2o_t = h2o_treat_base.query(where="FacilityName = '" + str(f.replace("'","''")) + "'").df
                    h2o_src = h2o_t.PrimaryWaterSource[0]
                    if str(h2o_src) == 'None':
                        h2o_src = ' No data. '
                    h2o_op = h2o_t.EntityName[0]
                    h2o_res = h2o_t.ResidentsServed[0]
                    if str(h2o_res) == 'nan':
                        h2o_res = (' No data. ')
                    h2o_con = h2o_t.WTSClassification[0]
                    h2o_ID = h2o_t.WaterTreatmentSystemID[0]
                    h2o_url = h2o_t.WaterTreatmentDataURL[0]
                    h2o_row = '<tr><td><span style="font-size:14px">' + str(f) + '</td><td>' + str(h2o_src) + '</td><td>' + str(h2o_res) + '</td><td><a href="' + str(h2o_url) + '">More info</a></td></span></tr>'
                h2o_t_tbl.append(h2o_row)
                h2o_t_tbl.append('</tbody></table>')
                embed_h2o_t = ''.join(h2o_t_tbl)
            else:
                embed_h2o_t = '<span style="font-size:14px"><i>Water treatment information is unavailable for this community.</i></span>'

            #Bulk Fuel
            bf_base = FeatureLayer('https://maps.commerce.alaska.gov/arcgis/rest/services/Bulk_Fuel/Bulk_Fuel_Inventory/FeatureServer/0')
            bf_query = bf_base.query(where="Community = '" + str(community.replace("'","''")) + "'").df
            if str(bf_query) != 'Empty DataFrame\nColumns: []\nIndex: []':
                bf_tbl = ['<table border="1"><th width="20%"><b>Owner</b></th><th width="15%"><b>Owner Contact</b></th><th width="10%"><b>Number of Tanks</b></th><th width="15%"><b>Gasoline Capacity</b></th><th width="15%"><b>Diesel Capacity</b></th><th width="20%"><b>Total Capacity</b></th><tbody>']
                for o in bf_query.OBJECTID:
                    tank_query = bf_base.query(where="OBJECTID = '" + str(o) + "'").df
                    number = tank_query.Number_of_Tanks[0]
                    owner_full = tank_query.Owner[0]
                    owner = owner_full.split('\n')[0]
                    own_contact = tank_query.Owner_Contact[0]
                    gas_cap = tank_query.Gasoline_Capacity[0]
                    dies_cap = tank_query.Diesel__Fuel_Capacity[0]
                    total_cap = tank_query.Total_Capacity_of_Tank[0]
                    bf_row = '<tr><td><span style="font-size:14px">' + str(owner) + '</span></td><td><span style="font-size:14px">' + str(own_contact) + '</span></td><td><span style="font-size:14px">' + str(number) + '</span></td><td><span style="font-size:14px">' + str(gas_cap) + '</span></td><td><span style="font-size:14px">' + str(dies_cap) + '</span></td><td><span style="font-size:14px">' + str(total_cap) + '</span></td></tr>'
                    bf_tbl.append(bf_row)
                bf_tbl.append('</tbody></table>')
                bf_tbl = ''.join(bf_tbl)
            else:
                bf_tbl = '<i>Bulk fuel inventory data is unavailable for this community.</i>'

            #Water distribution
            h2odist_base = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/Services/CDO_Utilities/MapServer/9')
            h2odist = h2odist_base.query(where=query + ' AND IsMostRecent = 1')
            if str(h2odist.df) != 'Empty DataFrame\nColumns: []\nIndex: []':
                h2odist_tbl = ['<table border="1"><th width="20%">Water Distribution Facility</th><th width="15%">Number of Residents Served</th><th width="15%">Number of Service Connections</th><th width="15%">Water Distribution System Class</th><th width="15%">Circulated or Heated System</th><th width="35%">Water Distribution System (AK Dept. of Environmental Conservation)</th><tbody>']
                facilities = h2odist.df.FacilityName
                for f in facilities:
                    h2odist = h2odist_base.query(where="FacilityName = '" + str(f.replace("'","''")) + "'").df
                    if str(h2odist.IsMostRecent[0]) == "1":
                        h2odist_op = '<span style="font-size:14px">' + str(h2odist.EntityName) + '</span>'
                        h2odist_res = '<span style="font-size:14px">' + str(h2odist.ResidentsServed[0]) + '</span>'
                        h2odist_svc = '<span style="font-size:14px">' + str(h2odist.NumberOfServiceConnections[0]) + '</span>'
                        h2odist_ht = '<span style="font-size:14px">' + str(h2odist.CirculatedOrHeatedSystem[0]) + '</span>'
                        h2odist_class = '<span style="font-size:14px">' + str(h2odist.WaterDistributionClassification[0]) + '</span>'
                        h2odist_ID = '<span style="font-size:14px">' + str(h2odist.WaterDistributionSystemsID[0]) + '</span>'
                        h2odistrow = '<tr><td><span style="font-size:14px">' + str(f) + '</span></td><td><span style="font-size:14px">' + str(h2odist_res) + '</span></td><td><span style="font-size:14px">' + str(h2odist_svc) + '</span></td><td><span style="font-size:14px">' + str(h2odist_class) + '</span></td><td><span style="font-size:14px">' + str(h2odist_ht) + "</span></td><td><span style='font-size:14px'>" + str(h2odist_ID) + "</span></td></tr>"
                        h2odist_tbl.append(h2odistrow)
                    h2odist_tbl.append('</tbody></table>')
                    embed_h2odist = ' '.join(h2odist_tbl)
            else:
                embed_h2odist = '<span style="font-size:14px"><i>Water distribution data is unavailable for this community.</i></span>'

            #Wastewater collection
            wasteh2o_base = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/Services/CDO_Utilities/MapServer/11')
            wasteh2o_c = wasteh2o_base.query(where=query,out_fields="*")
            if str(wasteh2o_c.df) != 'Empty DataFrame\nColumns: []\nIndex: []':
                print('Building wastewater collection table...')
                wasteh2o_c_tbl = ['<table border="1"><th width="25%">Wastewater System Facility</th><th width="20%">Number of Residents Served</th><th width="20%">Number of Service Connections</th><th width="15%">Wastewater System Class</th><th width="25%">Number of Lift Stations</th><tbody>']
                facilities = wasteh2o_c.df.FacilityName
                for f in list(set(facilities)):
                    wasteh2o_c = wasteh2o_base.query(where="FacilityName = '" + str(f.replace("'","''")) + "'").df
                    wasteh2o_op = wasteh2o_c.EntityName[0]
                    wasteh2o_svc = wasteh2o_c.NumberOfServiceConnections[0]
                    wasteh2o_res = wasteh2o_c.NumberOfResidentsServed[0]
                    wasteh2o_class = wasteh2o_c.ClassOfSewageCollectionFacility[0]
                    wasteh2o_lift = wasteh2o_c.NumberOfLiftStations[0]
                    #wasteh2o_url = wasteh2o_c.WastewaterCollectionURL[0]
                    wasteh2o_row = '<tr><td><span style="font-size:14px">' + str(f) + '</span></td><td><span style="font-size:14px">' + str(wasteh2o_res) + '</span></td><td><span style="font-size:14px">' + str(wasteh2o_svc) + '</span></td><td><span style="font-size:14px">' + str(wasteh2o_class) + '</span></td><td><span style="font-size:14px">' + str(wasteh2o_lift) + '</span></td></tr>'
                    wasteh2o_c_tbl.append(wasteh2o_row)
                wasteh2o_c_tbl.append('</tbody></table>')
                embed_wasteh2o_c = ' '.join(wasteh2o_c_tbl)
            else:
                embed_wasteh2o_c = '<span style="font-size:14px"><i>Wastewater collection information is unavailable to this community</i></span>'

            #Wastewater treatment
            wasteh2o_t_base = FeatureLayer('http://maps.commerce.alaska.gov/arcgis/rest/services/Services/CDO_Utilities/MapServer/10')
            wasteh2o_t = wasteh2o_t_base.query(where=query + " AND DataYear = 2018", out_fields = "*")
            if str(wasteh2o_t.df) != 'Empty DataFrame\nColumns: []\nIndex: []':
                wasteh2o_t_tbl = ['<table border="1"><th width="30%">Wastewater System Facility</th><th width="20%">Residents Served by Wastewater System</th><th width="20%">Number of Service Connections</th><th width="20%">Wastewater Treatment System ID (AK Dept. of Environmental Conservation)</th>']
                facilities = wasteh2o_t.df.FacilityName
                for f in list(set(facilities)):
                    print(f)
                    wasteh2o_t = wasteh2o_t_base.query(where="FacilityName = '" + str(f.replace("'","''")) + "'").df
                    wasteh2o_t_op = wasteh2o_t.EntityName[0]
                    wasteh2o_t_res = wasteh2o_t.NumberofResidentsServedByWWT[0]
                    wasteh2o_t_svc = wasteh2o_t.NumberOfServiceConnectionsToWWT[0]
                    wasteh2o_t_ID = wasteh2o_t.WastewaterTreatmentSystemID[0]
                    wasteh2o_t_row = '<tr><td><span style="font-size:14px">' + str(f) + '</span></td><td><span style="font-size:14px">' + str(wasteh2o_t_res) + '</span></td><td><span style="font-size:14px">' + str(wasteh2o_t_svc) + '</span></td><td><span style="font-size:14px">' + str(wasteh2o_t_ID) + '</span></td></tr>'
                    wasteh2o_t_tbl.append(wasteh2o_t_row)
                wasteh2o_t_tbl.append('</tbody></table>')
                embed_wasteh2o_t = ' '.join(wasteh2o_t_tbl)
            else:
                embed_wasteh2o_t = '<span style="font-size:14px"><i>Wastewater treatment data is currently unavailable for this community.</i></span>'

            panel_util["content"] = "<p><style type=\"text/css\">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 3px 8px;\n     border-radius: 5px;\n</style>\n</p>\n<p><a class=\"dcra-blue\" data-storymaps=\"MJ-ACTION-1524785467480\" data-storymaps-type=\"navigate\">Back to Table of Contents</a><br><div><p><b><u>Electric:</u></b></p><p>" + str(elec_tbl) + "</p><p>&nbsp;</p><p>&nbsp;</p><p><span style=\"font-size:12px\">Note: The <a href=\"http://www.akenergyauthority.org/Programs/PCE\" target=\"_blank\">Power Cost Equalization (PCE) program</a> provides rural Alaskans with economic assistance to offset higher energy costs as compared to urban areas. The electric information above includes information about how this program has supported this community, if applicable.</span></p><p>&nbsp;</p><p><b><u>Fuel Prices:</u></b></p><p>" + str(fuel_tbl) + "</p><p>&nbsp;</p><p><b><u>Bulk Fuel Inventory:</u></b></p><p>" + str(bf_tbl) + "</p><p>For more information about bulk fuel, visit the <span style='font-size:14px'><a href='https://dcced.maps.arcgis.com/apps/MapSeries/index.html?appid=9a5d6441c3134dc3b4322e5d45b24a7f' target='_blank'>Bulk Fuel Inventory Application</a>.</span></p><p>&nbsp;</p><p><b><u>Water Treatment: </u></b></p><p>" + str(embed_h2o_t) + "</p><p>&nbsp;</p><p><b><u>Water Distribution: </u></b></p><p>" + str(embed_h2odist) + "</p><p>&nbsp;</p><p><b><u>Wastewater Treatment: </u></b></p><p>" + str(embed_wasteh2o_t) + "</p>"
            print("Utilities panel content updated.")
            csmap.save()
        else:
            print("-------Utilities section is not on this storymap!!!----------")
            print(panel_util["title"])
            continue